In [ ]:
import pandas as pd
df=pd.read_csv('features_dwt.csv')

In [ ]:
# Normalize EEG data (110 columns)
eeg_data = df.iloc[:, 7:14].values  # Assuming columns 7 to 116 are EEG data
scaler_eeg = MinMaxScaler()
eeg_data_normalized = scaler_eeg.fit_transform(eeg_data)
static_features = df.iloc[:, :7].values
# Normalize static features
scaler_static = MinMaxScaler()
static_features_normalized = scaler_static.fit_transform(static_features)
# Combine normalized static features, normalized EEG data, and CPC values
normalized_data = np.concatenate((static_features_normalized, eeg_data_normalized), axis=1)

# Convert CPC values to one-hot encoded labels
one_hot_labels = to_categorical(cpc_values - 1, num_classes=5)  # Assuming CPC values start from 1
# Train-test split
train_data, test_data, train_labels, test_labels = train_test_split(
    normalized_data, one_hot_labels, test_size=0.2, random_state=42
)

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
names = ["Nearest Neighbors", "Linear SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "AdaBoost",
         "Naive Bayes"]
classifiers = [
    KNeighborsClassifier(5),
    SVC(kernel="linear", C=0.025),
    GaussianProcessClassifier(1.0 * RBF(1.0), warm_start=True),
    DecisionTreeClassifier(max_depth=3),
    RandomForestClassifier(max_depth=3, n_estimators=10),
    AdaBoostClassifier(),
    ]


In [ ]:
testlab=[]
for i in test_labels.tolist():
  testlab.append([i.index(1.0)+1])
trainlab=[]
for i in train_labels.tolist():
  trainlab.append([i.index(1.0)+1])

In [ ]:
# Define model
# Define model
model1 = Sequential()
model1.add(LSTM(units=128, input_shape=(14, 1), return_sequences=True))  # Input shape should match the number of EEG columns (117)
model1.add(Dropout(0.3))  # Add dropout layer with a dropout rate of 30%
model1.add(LSTM(units=64, return_sequences=True))
model1.add(LSTM(units=32))
model1.add(Dense(units=16, activation='relu'))
model1.add(Dense(units=5, activation='softmax'))

# Compile model
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#dwt-norm
model1.fit(x=train_data, y=train_labels, epochs=30, batch_size=32, validation_split=0.2)

Epoch 1/30
13/13 [==============================] - 9s 159ms/step - loss: 1.4645 - accuracy: 0.5556 - val_loss: 1.0808 - val_accuracy: 0.6082
Epoch 2/30
13/13 [==============================] - 1s 44ms/step - loss: 1.0745 - accuracy: 0.5788 - val_loss: 1.0218 - val_accuracy: 0.6082
Epoch 3/30
13/13 [==============================] - 1s 44ms/step - loss: 1.0542 - accuracy: 0.5788 - val_loss: 1.0211 - val_accuracy: 0.6082
Epoch 4/30
13/13 [==============================] - 1s 44ms/step - loss: 1.0490 - accuracy: 0.5788 - val_loss: 1.0139 - val_accuracy: 0.6082
Epoch 5/30
13/13 [==============================] - 1s 43ms/step - loss: 1.0481 - accuracy: 0.5788 - val_loss: 1.0101 - val_accuracy: 0.6082
Epoch 6/30
13/13 [==============================] - 1s 45ms/step - loss: 1.0458 - accuracy: 0.5788 - val_loss: 1.0058 - val_accuracy: 0.6082
Epoch 7/30
13/13 [==============================] - 1s 56ms/step - loss: 1.0433 - accuracy: 0.5788 - val_loss: 1.0046 - val_accuracy: 0.6082
Epoch 8/30
1

In [ ]:
loss, accuracy = model1.evaluate(x=test_data, y=test_labels)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

4/4 [==============================] - 0s 14ms/step - loss: 1.0209 - accuracy: 0.5738
Test Loss: 1.0209163427352905, Test Accuracy: 0.5737704634666443


In [ ]:
import numpy as np
import math
from math import exp
from sklearn.metrics import accuracy_score
data = pd.read_csv('features_all.csv')
data = data.dropna()
X = data.iloc[:, 1:-1].values
y = data['CPC'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

gem_matrix_train = np.zeros((X_train_scaled.shape[0], X_train_scaled.shape[0]))
for i in range(X_train_scaled.shape[0]):
    for j in range(i, X_train_scaled.shape[0]):
        dtw_distance = np.sqrt(np.sum(np.square(X_train_scaled[i] - X_train_scaled[j])))
        gem_matrix_train[i, j] = exp(-dtw_distance ** 2)
        gem_matrix_train[j, i] = gem_matrix_train[i, j]

gem_matrix_test = np.zeros((X_test_scaled.shape[0], X_train_scaled.shape[0]))
for i in range(X_test_scaled.shape[0]):
    for j in range(X_train_scaled.shape[0]):
        dtw_distance = np.sqrt(np.sum(np.square(X_test_scaled[i] - X_train_scaled[j])))
        gem_matrix_test[i, j] = exp(-dtw_distance ** 2)

clf = SVC(kernel='precomputed')
clf.fit(gem_matrix_train, y_train)
y_pred = clf.predict(gem_matrix_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")


Accuracy: 0.58


In [ ]:
#dwt
# stacked generalization with linear meta model on blobs dataset
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from keras.models import load_model
from keras.utils import to_categorical
from numpy import dstack
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
members=[model1,clf]
# load models from file
'''def load_all_models(n_models):
	all_models = list()
	for i in range(n_models):
		# define filename for this ensemble
		filename = 'models/model_' + str(i + 1) + '.h5'
		# load model from file
		model = load_model(filename)
		# add to list of members
		all_models.append(model)
		print('>loaded %s' % filename)
	return all_models'''

# create stacked model input dataset as outputs from the ensemble
def stacked_dataset(members, inputX):
  stackX = None
  for model in members:
    # make prediction
    if(model!=clf):
      yhat = model.predict(inputX)
      print('yhat: ',yhat)
      print(type(yhat))
    else:
      gem_matrix_test = np.zeros((inputX.shape[0], train_data.shape[0]))
      for i in range(inputX.shape[0]):
          for j in range(train_data.shape[0]):
              dtw_distance = np.sqrt(np.sum(np.square(inputX[i] - train_data[j])))
              gem_matrix_test[i, j] = exp(-dtw_distance ** 2)
      yhat = model.predict(gem_matrix_test)
      yhat1=[]
      for i in yhat.tolist():
        l=[0.0,0.0,0.0,0.0,0.0]
        l[i-1]=1.0
        yhat1.append(l)
      yhat=np.array(yhat1)
      print('yhat: ',yhat)
      print(type(yhat))
    # stack predictions into [rows, members, probabilities]
    if stackX is None:
      stackX = yhat
    else:
      stackX = dstack((stackX, yhat))
	# flatten predictions to [rows, members x probabilities]
  stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
  return stackX

#[1 0 0 0 0]
#[0 1 0 0 0]

# fit a model based on the outputs from the ensemble members
def fit_stacked_model(members, inputX, inputy):
  # create dataset using ensemble
  stackedX = stacked_dataset(members, inputX)
  print('stacked: ',stackedX)
  # fit standalone model
  #model = LogisticRegression()
  #model=clf
  model=classifiers[5]
  #odel=knn1
  #model=rfmodel ----->give 100 percent accuracy
  model.fit(stackedX, inputy)
  return model

# make a prediction with the stacked model
def stacked_prediction(members, model, inputX):
	# create dataset using ensemble
	stackedX = stacked_dataset(members, inputX)
	# make a prediction
	yhat = model.predict(stackedX)
	return yhat

# fit stacked model using the ensemble
model_final = fit_stacked_model(members, test_data, testlab)
# evaluate model on test set
yhat = stacked_prediction(members, model_final, test_data)
acc = accuracy_score(testlab, yhat)
print('Stacked Test Accuracy: %.3f' % acc)



4/4 [==============================] - 0s 12ms/step
yhat:  [[0.24669482 0.06660986 0.02884811 0.02231034 0.6355369 ]
 [0.24643439 0.0665826  0.02877615 0.02228015 0.63592666]
 [0.28646728 0.07420335 0.03961238 0.02599846 0.5737185 ]
 [0.24665868 0.06661652 0.02883951 0.02230841 0.63557684]
 [0.24657778 0.06660117 0.02880467 0.02228697 0.63572943]
 [0.30891827 0.07793819 0.04683165 0.02803066 0.53828126]
 [0.30889007 0.07795057 0.04682192 0.02802959 0.5383078 ]
 [0.2467255  0.06663075 0.02886672 0.02232483 0.6354523 ]
 [0.32940876 0.08120795 0.05438764 0.02992846 0.50506717]
 [0.24661928 0.06658425 0.02881814 0.02229161 0.6356868 ]
 [0.24666278 0.06657838 0.02882165 0.02228788 0.63564944]
 [0.24662319 0.0665893  0.0288168  0.02229029 0.6356803 ]
 [0.24686645 0.06672543 0.02895207 0.02239106 0.63506496]
 [0.24681208 0.06668508 0.02891739 0.02236355 0.63522196]
 [0.2467527  0.06664326 0.02888142 0.02233546 0.6353872 ]
 [0.26464844 0.0702781  0.03347512 0.02404634 0.60755193]
 [0.30891067 

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


4/4 [==============================] - 0s 18ms/step
yhat:  [[0.24669482 0.06660986 0.02884811 0.02231034 0.6355369 ]
 [0.24643439 0.0665826  0.02877615 0.02228015 0.63592666]
 [0.28646728 0.07420335 0.03961238 0.02599846 0.5737185 ]
 [0.24665868 0.06661652 0.02883951 0.02230841 0.63557684]
 [0.24657778 0.06660117 0.02880467 0.02228697 0.63572943]
 [0.30891827 0.07793819 0.04683165 0.02803066 0.53828126]
 [0.30889007 0.07795057 0.04682192 0.02802959 0.5383078 ]
 [0.2467255  0.06663075 0.02886672 0.02232483 0.6354523 ]
 [0.32940876 0.08120795 0.05438764 0.02992846 0.50506717]
 [0.24661928 0.06658425 0.02881814 0.02229161 0.6356868 ]
 [0.24666278 0.06657838 0.02882165 0.02228788 0.63564944]
 [0.24662319 0.0665893  0.0288168  0.02229029 0.6356803 ]
 [0.24686645 0.06672543 0.02895207 0.02239106 0.63506496]
 [0.24681208 0.06668508 0.02891739 0.02236355 0.63522196]
 [0.2467527  0.06664326 0.02888142 0.02233546 0.6353872 ]
 [0.26464844 0.0702781  0.03347512 0.02404634 0.60755193]
 [0.30891067 

In [ ]:
import sklearn
print(sklearn.metrics.mean_squared_error(testlab,yhat))
print(sklearn.metrics.mean_absolute_error(testlab,yhat))
print(sklearn.metrics.f1_score(testlab,yhat,average='weighted'))

0.13114754098360656
0.08196721311475409
0.9200198708395431
